In [1]:
import numpy as np
#import tensorflow as tf
#from tensorflow.python.client import device_lib
import pandas as pd
from zipfile import ZipFile
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
#from skimage import exposure
#from skimage.color import rgb2gray
import platform
import sys
from datetime import datetime
%matplotlib inline

In [2]:
driving = "driving_01"

# read CSV file with steering angle
csvdata = pd.read_csv("./data/" + driving + ".csv", header=None)

# 
# Images are in a Zip file, easier to upload to github
# 
with ZipFile("./data/" + driving + ".zip") as f:
    image_list = f.namelist()
    # open first image to get pixel size
    with f.open(image_list[1]) as fimage:
        tmp = mpimg.imread(fimage, format='jpeg')
    shape = (csvdata.shape[0], tmp.shape[0], tmp.shape[1], tmp.shape[2])
    X_data = np.zeros(shape, np.float)
    y_data = np.zeros(csvdata.shape[0], np.float)
    for k in range(1, len(image_list)):
        image = image_list[k]
        if image[11:17] == 'center':
            #im = Image.open("./data/" + image)
            X_data[k-1, :, :, :] = mpimg.imread(f.open(image), format='jpeg')


y_data = y_data + csvdata.iloc[:, 3]
print("Numpy data:", X_data.shape)
print(X_data.shape[0], "images of", X_data.shape[1], "by", X_data.shape[2], "RGB")

Numpy data: (70, 160, 320, 3)
70 images of 160 by 320 RGB


In [14]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.33)
print(X_train.shape)


(46, 160, 320, 3)


continue

Could use sample data (provided) as test:

In [15]:
#testdata = pd.read_csv("./sample-data/driving.csv", header=None)
#y_test = y_test + testdata.iloc[:, 3]


# Model Architecture

The proposed model has 5 convolutional layers and 4 fully connected.

In [9]:
from keras.models import load_model, Sequential
from keras.layers import Dense, Flatten, Conv2D
from keras.callbacks import EarlyStopping
from keras import __version__ as keras_version
import h5py

In [22]:
from keras.utils.np_utils import to_categorical
#y_binary = to_categorical(y_train)
#print(y_binary.shape, y_binary.dtype)

(46, 1) float64


In [12]:
model = Sequential()

# layer 1 - convolutional
model.add(Conv2D(512, (3, 3), activation='relu', input_shape=(X_data.shape[1], X_data.shape[2], X_data.shape[3])))

# layer 2
#model.add(Conv2D(256, (3, 3), activation='relu'))

# layer 3
#model.add(Conv2D(128, (3, 3), activation='relu'))

# layer 4
model.add(Conv2D(64, (3, 3), activation='relu'))

# layer 5
model.add(Conv2D(32, (3, 3), activation='relu'))

#model.add(Dropout(0.5))
model.add(Flatten())

# layer 6
#model.add(Dense(10000, activation='relu'))
#model.add(Dropout(0.5))

# layer 7
#model.add(Dense(1000, activation='relu'))

# layer 8
model.add(Dense(100, activation='relu'))

# layer 9
model.add(Dense(1, activation='relu'))

In [13]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 158, 318, 512)     14336     
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 156, 316, 256)     1179904   
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 154, 314, 128)     295040    
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 152, 312, 64)      73792     
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 150, 310, 32)      18464     
_________________________________________________________________
flatten_2 (Flatten)          (None, 1488000)           0         
_________________________________________________________________
dense_5 (Dense)              (None, 10000)             1488001000
__________

In [ ]:
ee = 1e-4
cb_earlystop = EarlyStopping(min_delta=ee, patience=3, verbose=1)

model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X_train, y_train, batch_size=32, epochs=10, callbacks=[cb_earlystop], verbose=1)


Epoch 1/10


## Evaluate

In [ ]:
score = model.evaluate(x_test, y_test, batch_size=32)
print(score)